In [87]:
# -*- coding: utf-8 -*-

"""Garmin fitfile pipeline

This notebook will exist to experiment around with the garmin
fit files.

Example:
        
Attributes:
    
Todo:
  * finish curvature
  * explain the garmin sampling algorithm
  * explore metadata from fitfile for:
    * device information
    * device settings
  * join in weather
    
"""

import sys
import fitparse
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
fn = '/Users/sentient-asoellinger/code/garmin_fun/examples/2628314301.fit'
ff = fitparse.FitFile(fn)

In [3]:
a = ff.get_messages('record')

In [61]:
def compute_num_secs(a, b):
    if pd.isnull(a) or pd.isnull(b): return pd.np.nan
    temp = a - b
    return temp.total_seconds() 

def poly_area(x,y):
    
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def menger_curvature(x, y):
    X = np.array(x, y, np.ones(len(x)))
    area = 0.5 * np.abs(np.linalg.det(X))
    num = 4 * area
    den = 

In [84]:
data = []

for record in ff.get_messages('record'):

    data.append(record.get_values())

data = pd.DataFrame(data)
data.index = data.timestamp
data['ts_offset_1'] = data.timestamp.shift(1)
data['period_seconds'] =  data.apply(lambda x: compute_num_secs(x['timestamp'], x['ts_offset_1']), axis=1)
data['position_lat_-1'] = data.position_lat.shift(1)
data['position_long_-1'] = data.position_long.shift(1)
data['position_lat_-2'] = data.position_lat.shift(2)
data['position_long_-2'] = data.position_long.shift(2)

In [101]:
gdf = gpd.GeoDataFrame(data)
gdf['coordinate_0'] = gdf.apply(lambda x: Point(x['position_lat'], 
                                                x['position_long']), axis=1)

gdf['coordinate_1'] = gdf.apply(lambda x: Point(x['position_lat_-1'], 
                                                x['position_long_-1']), axis=1)

gdf['coordinate_2'] = gdf.apply(lambda x: Point(x['position_lat_-2'], 
                                                x['position_long_-2']), axis=1)

In [106]:
gdf[['coordinate_0', 'coordinate_1', 'coordinate_2']].apply()

In [80]:
data1 = data.resample('1min').agg(['mean', 'std', 'min', 'max', 'count', 'skew'])
data1.columns = ["_".join(x) for x in data1.columns.ravel()]
count_cols = [col for col in data1.columns if 'count' in col]
data1 = data1.drop(count_cols[1:], axis=1)
data1.rename(columns={count_cols[0]:'count'}, inplace=True)
data1.columns
data1['distance_offset_1'] = data1.distance_mean.shift(1)
data1['distance_traveled'] = data1.distance_mean - data1.distance_offset_1
#data.period.describe()

In [82]:
data1.to_csv('tst.csv')